## 50 DATA PROCESSING

In [1]:
# If not installed, you may need:
# !pip install pydub gspread oauth2client yt-dlp

import os
import time
import shutil
import subprocess
from pydub import AudioSegment

# Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

##################################################
# A) GOOGLE SHEET CONFIG & AUTH
##################################################

SERVICE_ACCOUNT_JSON = os.path.join("data", "quantummusic-8dbce27ed321.json")
SCOPE = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_JSON, scopes=SCOPE)
gc = gspread.authorize(creds)

SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1oD1Oyp5inbcog9eUQTl8xnmYi6y-ZMln8Iaxz9xHEtY/edit#gid=780967666"
workbook = gc.open_by_url(SPREADSHEET_URL)

metadata_sheet       = workbook.worksheet("metadata")
tempogroups_sheet    = workbook.worksheet("tempogroups")
outputchunks_sheet   = workbook.worksheet("outputchunks")

##################################################
# B) HELPER: Find column index by name
##################################################
def find_col_index(col_name, header_list):
    """
    Locate column index (0-based) for 'col_name' in 'header_list' (case-insensitive).
    Returns None if not found.
    """
    col_name_lower = col_name.lower()
    for i, h in enumerate(header_list):
        if h.lower() == col_name_lower:
            return i
    return None


##################################################
# C) FIRST PASS: Download from YouTube if Processed = -1
##################################################
def first_pass_downloads():
    print("=== FIRST PASS: Downloading from YouTube for rows with Processed = -1 ===")

    # 1) Read entire "metadata" sheet
    all_values = metadata_sheet.get_all_values()
    if not all_values:
        print("No data in 'metadata' sheet. Nothing to do.")
        return  # no data

    header = all_values[0]
    data_rows = all_values[1:]

    # Identify relevant columns
    tracksource_idx = find_col_index("TrackSource", header)
    processed_idx   = find_col_index("Processed", header)
    filename_idx    = find_col_index("FileName", header)

    if tracksource_idx is None or processed_idx is None or filename_idx is None:
        raise ValueError("Could not find 'TrackSource', 'FileName', or 'Processed' columns in metadata header.")

    # 2) Collect rows where Processed = -1
    rows_to_download = []
    for i, row in enumerate(data_rows):
        sheet_row = i + 2  # 1-based offset for header
        if len(row) <= max(tracksource_idx, processed_idx, filename_idx):
            continue
        
        tracksource_val = row[tracksource_idx].strip()
        processed_val   = row[processed_idx].strip()
        file_name_in_sheet = row[filename_idx].strip()

        if processed_val == "-1" and tracksource_val:
            rows_to_download.append((sheet_row, tracksource_val, file_name_in_sheet))

    print(f"Found {len(rows_to_download)} YouTube URLs to download (Processed=-1).")

    download_folder = os.path.join("data", "rawunprocessed")
    os.makedirs(download_folder, exist_ok=True)

    # We'll store final statuses in a dict: row -> new_processed_val
    status_updates = {}

    for (sheet_row_number, youtube_url, file_name_in_sheet) in rows_to_download:
        print(f"Downloading row {sheet_row_number}, URL: {youtube_url}")
        
        # Ensure the final filename ends with .mp3
        # (If user already put .mp3, we won't double-append it.)
        if not file_name_in_sheet.lower().endswith(".mp3"):
            file_name_in_sheet += ".mp3"

        # Full path for output
        output_path = os.path.join(download_folder, file_name_in_sheet)

        cmd = [
            "yt-dlp",
            "-f", "bestaudio/best",
            "--extract-audio",
            "--audio-format", "mp3",
            "--audio-quality", "320K",
            "-o", output_path,
            youtube_url
        ]
        try:
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode == 0:
                print("Download succeeded.")
                # Mark processed=0 so it can be chunked in second pass
                status_updates[sheet_row_number] = "11"
            else:
                print(f"Download error (returncode={result.returncode}): {result.stderr}")
                # Mark processed=-2 to indicate error
                status_updates[sheet_row_number] = "-2"
        except Exception as e:
            print(f"Exception in download: {e}")
            status_updates[sheet_row_number] = "-2"
        
        # Wait 10 seconds before next
        time.sleep(10)
    
    # 3) Batch update sheet for these rows
    if status_updates:
        col_number = processed_idx + 1  # 1-based for Sheets
        cell_updates = []
        for row_idx, new_val in status_updates.items():
            cell_updates.append({
                'range': f"R{row_idx}C{col_number}",
                'values': [[new_val]]
            })
        metadata_sheet.batch_update(cell_updates)
        print(f"Updated {len(cell_updates)} rows in 'metadata' after YouTube downloads.")


##################################################
# D) SECOND PASS: Chunk if Processed = 0
##################################################

def second_pass_chunking():
    print("=== SECOND PASS: Chunking files for rows with Processed = 0 ===")

    # 1) Re-read "metadata" to see the updated states
    metadata_all_values = metadata_sheet.get_all_values()
    if not metadata_all_values:
        print("No data in 'metadata' sheet.")
        return

    metadata_header = metadata_all_values[0]
    metadata_data_rows = metadata_all_values[1:] if len(metadata_all_values) > 1 else []

    file_col_idx = find_col_index("FileName", metadata_header)
    processed_col_idx = find_col_index("Processed", metadata_header)
    if file_col_idx is None or processed_col_idx is None:
        raise ValueError("Could not find 'FileName' or 'Processed' column in metadata header.")

    # Build list of (sheet_row_number, file_name_in_sheet) where processed=0
    unprocessed_entries = []
    for i, row in enumerate(metadata_data_rows):
        sheet_row_number = i + 2
        if len(row) <= max(file_col_idx, processed_col_idx):
            continue
        
        file_name_in_sheet = row[file_col_idx].strip()
        processed_val = row[processed_col_idx].strip()
        if processed_val == "0":
            unprocessed_entries.append((sheet_row_number, file_name_in_sheet))

    print(f"Found {len(unprocessed_entries)} files marked Processed=0 in metadata for chunking.")

    # 2) Build set of base names for these
    unprocessed_base_names = set()
    for _, file_name_in_sheet in unprocessed_entries:
        base_name = os.path.splitext(file_name_in_sheet)[0]
        unprocessed_base_names.add(base_name)

    # 3) Read tempogroups, store only for these base names
    tempogroups_all = tempogroups_sheet.get_all_records()
    tempo_dict = {}
    for row in tempogroups_all:
        fname = row.get("FileName", "").strip()
        base_ = os.path.splitext(fname)[0]
        if base_ not in unprocessed_base_names:
            continue
        start_dur = float(row.get("StartDuration", 0))
        end_dur   = float(row.get("EndDuration", 0))
        tempo     = str(row.get("Tempo", "Unknown"))
        if base_ not in tempo_dict:
            tempo_dict[base_] = []
        tempo_dict[base_].append({
            "start": start_dur,
            "end": end_dur,
            "tempo": tempo
        })

    # Folders & settings
    data_folder      = "data"
    input_folder     = os.path.join(data_folder, "rawunprocessed")
    output_folder    = os.path.join(data_folder, "trainingdata")
    processed_folder = os.path.join(data_folder, "rawprocessed")
    error_folder     = os.path.join(data_folder, "rawerror")

    target_sample_rate = 44100
    chunk_duration_sec = 20

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(error_folder, exist_ok=True)

    # Helper functions
    def process_audio_file(file_path, is_mp3):
        """Load MP3 or WAV, convert to target SR mono."""
        if is_mp3:
            audio = AudioSegment.from_file(file_path, format="mp3")
        else:
            audio = AudioSegment.from_file(file_path, format="wav")
        audio = audio.set_frame_rate(target_sample_rate).set_channels(1)
        duration_sec = len(audio) / 1000.0
        return audio, duration_sec

    def chunk_audio_by_tempo(audio, base_name, file_name_sheet, tempo_info):
        rows_for_sheet = []
        for tinfo in tempo_info:
            tempo_start = float(tinfo["start"])
            tempo_end   = float(tinfo["end"])
            tempo_label = tinfo["tempo"]
            chunk_start_sec = tempo_start

            while chunk_start_sec < tempo_end:
                chunk_end_sec = chunk_start_sec + chunk_duration_sec
                if chunk_end_sec > tempo_end:
                    actual_length_sec = tempo_end - chunk_start_sec
                    padded_sec = chunk_duration_sec - actual_length_sec
                    start_ms = int(chunk_start_sec * 1000)
                    end_ms   = int(tempo_end * 1000)
                    audio_chunk = audio[start_ms:end_ms]
                    if padded_sec > 0:
                        silence_chunk = AudioSegment.silent(duration=int(padded_sec * 1000))
                        audio_chunk = audio_chunk + silence_chunk

                    chunk_filename = f"{base_name}_{int(chunk_start_sec)}s_{int(chunk_end_sec)}s_{tempo_label}_padded.wav"
                    padded_flag = "Y"
                else:
                    start_ms = int(chunk_start_sec * 1000)
                    end_ms   = int(chunk_end_sec * 1000)
                    audio_chunk = audio[start_ms:end_ms]
                    chunk_filename = f"{base_name}_{int(chunk_start_sec)}s_{int(chunk_end_sec)}s_{tempo_label}.wav"
                    padded_flag = "N"

                out_path = os.path.join(output_folder, chunk_filename)
                audio_chunk.export(out_path, format="wav")

                rows_for_sheet.append({
                    "FileName": file_name_sheet,
                    "ChunkFileName": chunk_filename,
                    "Start Duration": f"{chunk_start_sec:.2f}",
                    "End Duration": f"{min(chunk_end_sec, tempo_end):.2f}",
                    "Tempo": tempo_label,
                    "Padded": padded_flag
                })

                chunk_start_sec += chunk_duration_sec

        return rows_for_sheet

    all_output_rows = []
    status_updates_for_chunk = {}

    # 4) Loop over entries with Processed=0
    for (sheet_row_number, file_name_in_sheet) in unprocessed_entries:
        local_path = os.path.join(input_folder, file_name_in_sheet)
        lower_name = file_name_in_sheet.lower()

        if not os.path.exists(local_path):
            print(f"Error: File '{file_name_in_sheet}' not found in rawunprocessed. Skipping.")
            # Mark processed = -3
            status_updates_for_chunk[sheet_row_number] = "-3"
            continue

        if not (lower_name.endswith(".mp3") or lower_name.endswith(".wav")):
            print(f"Error: File '{file_name_in_sheet}' is not .mp3 or .wav. Skipping.")
            status_updates_for_chunk[sheet_row_number] = "-3"
            continue

        try:
            is_mp3 = lower_name.endswith(".mp3")
            audio, total_sec = process_audio_file(local_path, is_mp3)
            base_name = os.path.splitext(file_name_in_sheet)[0]
            if base_name in tempo_dict and tempo_dict[base_name]:
                tempo_info = tempo_dict[base_name]
            else:
                tempo_info = [{
                    "start": 0,
                    "end": total_sec,
                    "tempo": "Unknown"
                }]

            chunk_rows = chunk_audio_by_tempo(audio, base_name, file_name_in_sheet, tempo_info)
            all_output_rows.extend(chunk_rows)

            # Move original file to processed
            shutil.move(local_path, os.path.join(processed_folder, file_name_in_sheet))
            status_updates_for_chunk[sheet_row_number] = "1"  # success

        except Exception as e:
            print(f"Error processing {file_name_in_sheet}: {e}")
            if os.path.exists(local_path):
                shutil.move(local_path, os.path.join(error_folder, file_name_in_sheet))
            status_updates_for_chunk[sheet_row_number] = "-3"

    # 5) Single append to 'outputchunks'
    if all_output_rows:
        rows_to_append = []
        for row in all_output_rows:
            rows_to_append.append([
                row["FileName"],
                row["ChunkFileName"],
                row["Start Duration"],
                row["End Duration"],
                row["Tempo"],
                row["Padded"]
            ])
        outputchunks_sheet.append_rows(rows_to_append, value_input_option='RAW')
        print(f"Appended {len(rows_to_append)} rows to 'outputchunks' in second pass.")

    # 6) Batch update "Processed" for chunk results
    if status_updates_for_chunk:
        col_number = processed_col_idx + 1
        cell_updates = []
        for row_idx, new_val in status_updates_for_chunk.items():
            cell_updates.append({
                'range': f"R{row_idx}C{col_number}",
                'values': [[new_val]]
            })
        metadata_sheet.batch_update(cell_updates)
        print(f"Updated {len(cell_updates)} rows in 'metadata' after chunking pass.")


##################################################
# E) SINGLE CELL MAIN LOGIC
##################################################

def run_all_passes():
    """
    1) First pass: download from YouTube where Processed=-1, set them to 0 or -2
    2) Second pass: chunk audio where Processed=0, set them to 1 or -3
    """
    first_pass_downloads()
    second_pass_chunking()
    print("All passes completed.")

# Finally, just call run_all_passes():
if __name__ == "__main__":
    run_all_passes()

=== FIRST PASS: Downloading from YouTube for rows with Processed = -1 ===
Found 0 YouTube URLs to download (Processed=-1).
=== SECOND PASS: Chunking files for rows with Processed = 0 ===
Found 1104 files marked Processed=0 in metadata for chunking.
Error: File '.DS_Store_2' is not .mp3 or .wav. Skipping.
Appended 43959 rows to 'outputchunks' in second pass.
Updated 1104 rows in 'metadata' after chunking pass.
All passes completed.
